### Practice 3 - Brute force attempt to find the most likely sequence of hidden states

Markos Flavio B. G. O.

__Context: Hidden Markov Models (MMs).__

__Course: Unsupervised Machine Learning Hidden Markov Models in Python (Udemy, LazyProgrammer)__

This notebook implements a programming exercise that shows the inefficieny of trying to evaluate the most likely sequence of hidden states, given a sequence of observations, by brute force.
    
__Specific objectives__

     1. Build a brute-force algorithm that evaluates p(z, x) given x for each z and outputs the most likely z.

In [22]:
import numpy as np
from IPython.display import Image
import itertools

The algorithm should compute the most likely sequence of hidden states z for the given x. The joint probability of z and x is given by the formula below:

<img src="./Images/pzx.png" width="500">

In [85]:
def most_likely_seq_brute_force(A, B, pi, z, x, xt):
    '''
    Computes the most likely sequence of hidden states z from a sequence of observations xt.
    Parameters:
        A (ndarray): state-transition matrix with size (M, M).
        B (ndarray): emission probabilties vector with size (M, K)
        pi (ndarray): initial state distribution vector with size (K, 1)
        z (dicionary): links state names to indexes (size M)
        x (dicionary): links state names to indexes (size K)
        xt (list of strings): list of observations with size T.
    Returns:
        z_star (list): most likely sequence of states (size T).
    '''
    T = len(xt) # or len(x)
    M = len(z)
    K = len(x)

    # computing the indexes of observations
    xt_seq = [x[obs] for obs in xt]
    
    # generating all possible state sequences (M^T)
    st_seqs = [item for item in itertools.product(list(range(M)), repeat=T)]
    
    max_prob = 0; best_st_seq = None
    for st_seq in st_seqs: # for each sequence
        st_seq = list(st_seq)
        pzx = 0
        for i, (st_idx, xt_idx) in enumerate(zip(st_seq, xt_seq)):
            if i == 0:
                pzx = pi[st_idx]*B[st_idx, xt_idx]
            else:
                pzx = pzx*A[st_idx_old, st_idx]*B[st_idx, xt_idx]
            st_idx_old = st_idx
        if pzx > max_prob:
            best_st_seq = st_seq
            max_prob = pzx
    
    # return state names instead of indexes
    return [list(z.keys())[list(z.values()).index(st_idx)] for st_idx in best_st_seq]

Let's test algorithm using this simple example (taken from the course):
<img src="./Images/weather_example.png" width=300>

States are weathers (hot and cold), which we don't have access to. On the contrary, the decision of taking ice cream or not on days are the observations, which we do have access to.

The A matrix is defined in the diagram. Additionally, we state that:
 - B(ice cream | hot day) = 0.9
 - B(no ice cream | hot day) = 0.1
 - B(ice cream | cold day) = 0.2
 - B(no ice cream | cold day) = 0.8
 - pi(ice cream) = pi(no ice cream) = 0.5
 - 'ice cream' observation and 'hot day' state have both index 0. 

In [86]:
x = {"ice cream": 0, "no ice cream":1}
z = {"hot day": 0, "cold day": 1}
A = np.array([[0.9, 0.1], [0.1, 0.9]])
B = np.array([[0.9, 0.2], [0.1, 0.8]])
pi = np.array([0.5, 0.5])

Let's find out the most likely weather sequence for a simple sequence of decisions:

In [88]:
xt = ["ice cream", "ice cream", "ice cream", "ice cream", "ice cream"]
most_likely_seq_brute_force(A, B, pi, z, x, xt)

['hot day', 'hot day', 'hot day', 'hot day', 'hot day']

Let's now try a more complex and unintuitive sequence:

In [93]:
xt = ["ice cream", "no ice cream", "no ice cream", "no ice cream", "ice cream", "no ice cream", "no ice cream", "no ice cream"]
most_likely_seq_brute_force(A, B, pi, z, x, xt)

['hot day',
 'cold day',
 'cold day',
 'cold day',
 'cold day',
 'cold day',
 'cold day',
 'cold day']

The algorithm seems to be working. The problem with this force brute approach is it has to scan over all possible sequence of states (M^T). Also, for each sequence, the algorithm scans through all of the sequence elements (T), having a time complexity of O(T*M^T). This makes this naive approach unfeasible to handle complex real-world problems.